# Unit 5: Computer Algebra Systems and Symbolic Computation

$\newcommand{\RR}{\mathcal{R}}$
$\newcommand{\FF}{\mathbb{F}}$
$\newcommand{\QQ}{\mathbb{Q}}$
$\newcommand{\ZZ}{\mathbb{Z}}$
$\newcommand{\NN}{\mathbb{N}}$
$\newcommand{\PP}{\mathbb{P}}$
$\renewcommand{\cong}{\equiv}$
$\renewcommand{\mod}{\bmod}$

## Symbolic Computation

To compute *exactly* means no error is introduced during arithmetic --- something *not* true when working with floats.  In a computer algebra system `sqrt(2) = sqrt(2)` and not `sqrt(2) = 1.41\ldots` as one may expect.

Our interest is performing *exact* arithmetic in rings with efficient algorithms.  In particular we are interested in the: *ring of integers* 
$$\ZZ = \{0, -1, 1, 2, -2, 3, -3, \ldots \}$$
and *polynomials* 
$$\RR[x] = \left\{ \sum_{k=0}^{N} a_kx^k \, : \, k \in \NN,\, a_k \in \RR \right\}.$$

---
## Preliminaries
Let $\NN = \{0,1,2,\ldots\}$ be the set of *natural numbers*, $\ZZ = \{\ldots,-2,-1,0,1,2,\ldots\}$ be the *integers* and $\PP$ be the set of odd primes.

##### Definition (Ring)

A ring $\RR$ is a set with addtion $(+)$ and multiplciation $(\cdot)$ satisfying the following conditions called the *ring axioms*:

1. $(\RR,+)$ is an abelian group, that is:
    - $(+)$ is associative *and* commutative,
    - $\exists b \in \RR$ called the *additive identity* satisfying $\forall a \in \RR \; a + b = a$,
    - $\forall a \in \RR\; \exists b \in \RR$ called the *additive inverse* satisfying $a + b = 0$.
2.  $(\RR,\cdot)$ is a monoid, that is:
    -  $(\cdot)$ is associative, and
    -  $\exists b \in \RR$ called the *multiplicative identity* satisfying $\forall a \in \RR; \; a \cdot b = b \cdot a = a$
3.  Multiplication *distributes* over addition.  That is, $\forall a,b,c \in \RR$:
    - $a \cdot (b+c) = (a \cdot b) + (a \cdot c)$
    - $(b+c) \cdot a = (b \cdot a) + (c \cdot a)$
---

##### Definition (Field)
A ring $\RR$ is also a *field* when each non-zero element has a *multiplicative inverse*.  Equivalently, $\RR$ is a ring when
$$
\forall a \in \RR^{\neq 0};\; \exists b \in \RR \; : \; a \cdot b.
$$

For instance, the *rationals* $$\QQ = \left\{ \frac{a}{b} \,:\, a,b \in \ZZ, \, b \neq 0 \right\}$$
is a field because $\frac a b \cdot \frac b a = 1$ when $a,b \neq 0$.

---

##### Theorem (Division Algorithm)
Let $a,b \in \ZZ$ with $b > 0$.  There is *unique* $q$ and $r$ (called the *quotient* and *remainder*) satisfying
$$a = b \cdot q + r.$$

**Proof (Existence)**

In [1]:
# An example
a = 23
b = 7
q = a ÷ b # \div + [TAB]
@show q
@show Int(floor(a/b))

r = a % 3 
@show r;

q = 3
Int(floor(a / b)) = 3
r = 2


In [2]:
function remainder(a::T,b::T) where T <: Int #Note that later we'll extend this to integral domains
    a < 0 && return remainder(-a,b) #short circuit evalution  #later replace `0` with `zero(T)`
    a < b && return a
    return remainder(a-b,b)
end

remainder (generic function with 1 method)

In [3]:
#note there is an in-built `rem`
remainder(23,7), rem(23,7), 23 % 7, 23- (23 ÷ 7)*7

(2, 2, 2, 2)

In [4]:
remainder(15,5), remainder(4,101), remainder(125,4)

(0, 4, 1)

In [5]:
function quo(a::T,b::T) where T <: Int
    a < 0 && return -quo(-a,b)
    a < b && return 0
    return 1 + quo(a-b, b)
end

quo (generic function with 1 method)

In [6]:
quo(15,5), quo(4,101), quo(125,4)

(3, 0, 31)

In [7]:
for (a,b) in [(15,5), (4,101), (125,4)]
    q, r = quo(a, b), rem(a, b)
    @show (a, b, q, r, a == b * q + r)
end

(a, b, q, r, a == b * q + r) = (15, 5, 3, 0, true)
(a, b, q, r, a == b * q + r) = (4, 101, 0, 4, true)
(a, b, q, r, a == b * q + r) = (125, 4, 31, 1, true)


Of course, in Julia we have `%` and `÷` for remainder and  and quotient.

(Note that in Python `//` is the quotient whereas in Julia `//` defines a rational type).

In [8]:
2 // 3, float(2//3)

(2//3, 0.6666666666666666)

---
## Elementary Number Theory

The sets $\ZZ_n = \{0,1,\ldots,n-1\}$ for $n \in \NN$ are rings called *residue classes* when addition and multiplication are done 'mod n'.  This is sometimes called 'clock arithmetic' as three hours past eleven is two because $11 + 3 \cong 2 \mod 12$. 

We say/write 
$$a \cong b \mod c$$ 
when *$a$ is congruent to $b$ modulo $c$*.  Also:
$$a \cong b \mod c \iff \mathop{\textrm{rem}}(a,c) = b \iff \texttt{a % c = b}.$$



In [9]:
# We have already seen something similar with overflow, for example:
UInt8(253) + UInt8(4)

0x01

In [10]:
UInt8(16) * UInt8(17)

0x10

---

#### Example
$\ZZ_6 = \{0,\ldots,5\}$ has the following addition table

In [11]:
n = 6
Z_n = 0:(n-1)

0:5

In [12]:
typeof(Z_n)

UnitRange{Int64}

In [13]:
A = [(x+y) % n for y in Z_n, x in Z_n]

6×6 Matrix{Int64}:
 0  1  2  3  4  5
 1  2  3  4  5  0
 2  3  4  5  0  1
 3  4  5  0  1  2
 4  5  0  1  2  3
 5  0  1  2  3  4

Notice: 
 1.  the addition table is symmetric (equal to its transpose) which can only happen when the addition is commutative,
 1.  the additive identity is $0$,
 1.  each column (and row) has $0$ and thereby each element has an additive inverse: for instance, the third column indicates that $2$ has additive inverse $4$ and correspondingly $2 + 4 \cong 0 \mod 6$.

In [14]:
additive_inverses = [findfirst((A .== 0)[:,k+1])-1 for k in Z_n] #Can this be done more nicely?
println(additive_inverses)

[0, 5, 4, 3, 2, 1]


$\ZZ_6$ has the following multiplication table

In [15]:
M = [(x*y) % n for y in Z_n, x in Z_n]

6×6 Matrix{Int64}:
 0  0  0  0  0  0
 0  1  2  3  4  5
 0  2  4  0  2  4
 0  3  0  3  0  3
 0  4  2  0  4  2
 0  5  4  3  2  1

Notice:
1.  the multiplicative identity is $1$,
1.  not all rows contain one, which means there are some elements that do *not* have *multiplicative inverse*.  For instance there is no $b \in \ZZ_6$ such that $2 \cdot b \cong 1 \mod 6$.

In [16]:
println([(2*y) % n for y in Z_n])

[0, 2, 4, 0, 2, 4]


In [17]:
mult_inverses = Z_n[[sum((M .== 1)[:,k+1]) > 0 for k in Z_n]] #Can this be done more nicely?
println("Elements with a multiplicative inverse: ", mult_inverses )
println("Elements withoout a multiplicative inverse: ", setdiff(Z_n,mult_inverses))

Elements with a multiplicative inverse: [1, 5]
Elements withoout a multiplicative inverse: [0, 2, 3, 4]


---

Notice $\ZZ_6$ is *not* a field because, recall, there is no multiplicative inverse for $2$.  However, $\ZZ_p$ is a field when $p$ is a prime number.

##### Example

$\ZZ_5 = \{0,\ldots,5\}$ is a field and has the following multiplication table. 

In [18]:
n = 5
Z_n = 0:(n-1)

0:4

In [19]:
M = [(x*y) % n for y in Z_n, x in Z_n]

5×5 Matrix{Int64}:
 0  0  0  0  0
 0  1  2  3  4
 0  2  4  1  3
 0  3  1  4  2
 0  4  3  2  1

In [20]:
mult_inverses = Z_n[[sum((M .== 1)[:,k+1]) > 0 for k in Z_n]] #Can this be done more nicely?
println("Elements with a multiplicative inverse: ", mult_inverses )
println("Elements withoout a multiplicative inverse: ", setdiff(Z_n,mult_inverses))

Elements with a multiplicative inverse: [1, 2, 3, 4]
Elements withoout a multiplicative inverse: [0]


Notice every column (and row) contains $1$ asides the first column (and row) which corresponds to $0$.  For instance, the third column indicates the inverse of $3$ is $2$ and correspondingly $3 \cdot 2 \cong 1 \mod 5$.

---

### Greatest Common Divisor

The *greatest common divisor* is one of the fundamental operations of a computer algebra system.  For instance, we need `gcd` to reduce fractions to their canonical form.

##### Def (GCD)

Let $a,b \in \ZZ$ not both 0.  We say $g \in \ZZ$ is the *greatest common divsor* of $a$ and $b$, denoted $\gcd(a,b)$, when
1.  $g | a$ and $g | b$ ($g$ is a common divisor), 
1.  $h | a \;\wedge\; h | b \implies h | g$ (greatest),
1.  $g > 0$ (required for uniqueness) 

##### Example (GCD)
The $\gcd(6,4) = \gcd(2\cdot 3, 2 \cdot 2) = 2$.

---

In [21]:
#The is an in-built gcd() function for integers (and a few more types) 
#but we'll soon make our own
using Random; Random.seed!(0)
a_test = *(rand(1:100,5)...)
b_test = *(rand(1:100,5)...)
@show a_test, b_test
gcd(a_test, b_test)

(a_test, b_test) = (884473730, 11119680)


10

#### Euclid's Algorithm

The **Euclidean Algorithm** computes the gcd of two integers.  (Actually the Euclidean Algorithm computers gcds for any two elements of a *Euclidean Domain*).  The algorithm exploits the following property of the gcd:

##### Lemma
Let $a,b\in \ZZ$ and $a>0$, $b>0$ and $a = b\cdot q + r$ with $r \in [0,b)$.  Then
1.  $\gcd(b,a) = \gcd(a,b)$,
1.  $\gcd(a,b) = gcd(r,b)$,
1.  $\gcd(a,b) = \gcd(a-b,b)$.

Is is straightforward to turn this lemma into an algorithm:

In [22]:
function euclid_alg(a,b)
    (b == 0) && return a
    return euclid_alg(b, a % b)
end

euclid_alg(2, 2*2), euclid_alg(2*3, 3*7), euclid_alg(2*2, 13)

(2, 3, 1)

In [23]:
gcd(a_test, b_test), euclid_alg(a_test,b_test)

(10, 10)

--- 

#### Extended Euclid's Algorithm

The **Extended Euclidean Algorithm** in addition to the **gcd(a, b)**, also computes $s$ and $t$ such that
$$as + bt = gcd(a,b).$$

In [24]:
function ext_euclid_alg(a,b)
    a == 0 && return b, 0, 1
    g, s, t = ext_euclid_alg(b % a, a)
    return g, t - (b ÷ a)*s, s
end

pretty_print_egcd((a,b),(g,s,t)) = println("$a × $s + $b × $t = $g") #\times + [TAB]

for (a,b) in [(4,12), (9,12), (4,13)]
    pretty_print_egcd((a,b),ext_euclid_alg(a,b))
end

4 × 1 + 12 × 0 = 4
9 × -1 + 12 × 1 = 3
4 × -3 + 13 × 1 = 1


In [25]:
#Note there is an in-built gcdx - let's compare against that too
gcdx(a_test,b_test), ext_euclid_alg(a_test,b_test)

((10, 293957, -23381720), (10, 293957, -23381720))

--- 

### Inversion in $\ZZ_m$

Notice the Extended Euclidean Algorithm computes inverses in $\ZZ_m$.  Given $a \in \ZZ_m^{\neq 0}$ the $egcd(a,m)$ returns $s$ and $t$ such that
$$a \cdot s + m \cdot t = gcd(a,m).$$
Taking the entire equation $\mod m$ we get
$$a \cdot s + m \cdot t \cong \gcd(a,m) \mod m \implies a \cdot s + 0 \cong \gcd(a,m) \mod m \implies a \cdot s \cong \gcd(a,m) \mod m.$$
Provided $\gcd(a,m) = 1$ (i.e. "coprime" or "relatively prime") we get $a \cdot s \cong 1 \mod m$ and thus $a^{-1} \cong s \mod m$.

##### Example
To find the inverse of $5 \in \ZZ_{13}$ do:

In [33]:
a, m = 5, 13

(5, 13)

In [34]:
inverse_mod(a,m) = mod(ext_euclid_alg(a,m)[2],m);

In [35]:
i = inverse_mod(a,m)

8

In [36]:
mod(a*i,m)

1

---

---
### Chinese Remaindering

##### Problem Statement
Two integers $a$ and $b$ are *relatively prime* when $\gcd(a,b) = 1$.  Given a finite sequence of relatively prime integers $m_1,\ldots,m_k$ called the *moduli* and correspoing integers $u_1,\ldots,u_k$ called the *images*, find $u \in \ZZ$ such that
\begin{align*}
u &\cong u_1 \mod m_1 \\
u &\cong u_2 \mod m_2 \\
&\; \vdots \\
u &\cong u_k \mod m_k
\end{align*}

For example, given the following information
\begin{align*}
u &\cong 4 \mod 5 &\implies&& (u_1,\, m_1) &= (4,\, 5)\\
u &\cong 5 \mod 7 &\implies&& (u_2,\, m_2) &= (5,\, 7)
\end{align*}
we want to deduce $u = 19$ because

In [37]:
(19 % 5 == 4, 19 % 7 == 5)

(true, true)

---
##### Theorem (CRT)

Let $M = m_1 \cdot m_2 \cdots \cdot m_k$.  There exists a unique $u \in \ZZ$ on $\{0,\ldots,M-1\}$ such that $u \cong u_i \mod m_i$ for $i = 1,\ldots,k$.

**Proof of Existence**:

(This is a constructive proof which provides an algorithm for determining $u$).

Let $u = v_1 + v_2m_1 + v_3m_1m_2 + \cdots + v_{k+1} m_1 m_2 \cdots m_{k}$ and notice the vanishing terms after reducing $u$ modulo $m_k$:
$$
u \mod m_j = v_1 + v_2m_1 + \cdots + v_{j+1}m_1\cdots m_{j} + 0 + \cdots + 0
$$

We have
\begin{align*}
u_1 &\cong v_1 \mod m_1 &&\implies v_1 \gets u_1 \\
u_2 &\cong v_1 + v_2 m_1 \mod m_2 &&\implies v_2 \gets (u_2 - v_1) \cdot (m_1)^{-1} \mod m_2 \\
u_3 &\cong v_1 + v_2 m_1 + v_3 m_1 m_2 \mod m_3 &&\implies v_3 \gets (u_3-v_1-v_2 m_1)(m_1 m_2)^{-1} \mod m_3\\
&\;\,\vdots
\end{align*}

**Proof of Uniquenss**

Left as an exercise. :P

---

##### Example (CRT)

Suppose $u \cong 2 \mod 5$, $u \cong 1 \mod 7$, $u \cong 1 \mod 3$.  What is $u$?

Proceeding with CRT...

In [53]:
uᵢ = [2, 1, 1] # \_i + [TAB]
m = [5, 7, 3]
v = Vector{Int}(undef,3)

v[1] = uᵢ[1]
v[2] = (uᵢ[2] - v[1])*inverse_mod(m[1], m[2]) % m[2]
v[3] = (uᵢ[3] - v[1] - v[2]*m[1])*inverse_mod(m[1]*m[2], m[3]) % m[3]
@show v #intermediate 

u = v[1] + v[2]*m[1] + v[3]*m[1]*m[2]
if [u % m[1], u % m[2], u % m[3]] == uᵢ
    println("We have a solution, u = $u")
end

v = [2, -3, 1]
We have a solution, u = 22


---

## The Polynomial Ring $Q[x]$